In [1]:
import math
from datetime import datetime
from zoneinfo import ZoneInfo

# -------------------------------------------------
# 1. Day of year, declination, equation of time
# -------------------------------------------------

def day_of_year(d: datetime) -> int:
    return d.timetuple().tm_yday


def solar_declination_deg(n: int) -> float:
    """
    Computes solar declination (δ) in degrees.
    Cooper (1969) approximation:
        δ = 23.45 * sin( 360/365 * (284 + n) )
    """
    return 23.45 * math.sin(math.radians(360.0 * (284 + n) / 365.0))


def equation_of_time_min(n: int) -> float:
    """
    Computes equation of time (minutes), standard approximation.
    """
    B = math.radians(360.0 * (n - 81) / 364.0)
    return 9.87 * math.sin(2 * B) - 7.53 * math.cos(B) - 1.5 * math.sin(B)


# -------------------------------------------------
# 2. Solar zenith using UTC
# -------------------------------------------------

def solar_zenith_from_utc(lat_deg: float,
                          lon_deg: float,
                          when_utc: datetime) -> tuple[float, float, int]:
    """
    Computes solar zenith angle (deg) and cos(theta_z) from UTC time.

    Parameters
    ----------
    lat_deg : float        Latitude in degrees (N positive).
    lon_deg : float        Longitude in degrees (E positive, W negative).
    when_utc : datetime    Time in UTC, timezone-aware or naive-but-UTC.

    Returns
    -------
    theta_z_deg : float
    cos_theta_z : float
    n : int (day of year)
    """
    
    if when_utc.tzinfo is not None:
        when_utc = when_utc.astimezone(ZoneInfo("UTC"))

    n = day_of_year(when_utc)
    delta_deg = solar_declination_deg(n)
    eot = equation_of_time_min(n)  # minutes

    # UTC in decimal hours
    t_utc = when_utc.hour + when_utc.minute / 60.0 + when_utc.second / 3600.0

    # Local solar time (hours), east-longitude positive
    lst = t_utc + lon_deg / 15.0 + eot / 60.0

    # Hour angle (deg)
    omega_deg = 15.0 * (lst - 12.0)

    # Zenith angle
    phi = math.radians(lat_deg)
    delta = math.radians(delta_deg)
    omega = math.radians(omega_deg)

    cos_theta = (
        math.sin(phi) * math.sin(delta)
        + math.cos(phi) * math.cos(delta) * math.cos(omega)
    )
    cos_theta = max(min(cos_theta, 1.0), -1.0)  # clamp

    theta_deg = math.degrees(math.acos(cos_theta))
    return theta_deg, cos_theta, n


# -------------------------------------------------
# 3. Decompose GHI into DNI & DHI (Erbs corr.)
# -------------------------------------------------

def decompose_ghi_to_dni_dhi(ghi: float,
                             theta_z_deg: float,
                             n: int) -> dict:
    """
    Decomposes GHI into DNI and DHI using Erbs correlation.

    Parameters
    ----------
    ghi : float           Global horizontal irradiance (W/m^2).
    theta_z_deg : float   Solar zenith angle (degrees).
    n : int               Day of year.

    Returns
    -------
    dictionary with keys: DNI, DHI, kt, Fd, Gon, G0, cos_theta_z
    """
    G_sc = 1367.0  # Solar constant, W/m^2

    cos_theta = math.cos(math.radians(theta_z_deg))

    # If sun is below horizon, no direct solar beam available.
    if cos_theta <= 0:
        return {
            "DNI": 0.0,
            "DHI": ghi,
            "kt": None,
            "Fd": None,
            "Gon": None,
            "G0": None,
            "cos_theta_z": cos_theta,
        }

    # Extraterrestrial normal irradiance
    Gon = G_sc * (1.0 + 0.033 * math.cos(math.radians(360.0 * n / 365.0)))

    # Extraterrestrial horizontal irradiance
    G0 = Gon * cos_theta

    # Clearness index
    kt = ghi / G0

    # Erbs diffuse fraction
    if kt <= 0.22:
        Fd = 1.0 - 0.09 * kt
    elif kt <= 0.80:
        Fd = (
            0.9511
            - 0.1604 * kt
            + 4.388 * kt**2
            - 16.638 * kt**3
            + 12.336 * kt**4
        )
    else:
        Fd = 0.165

    Fd = max(min(Fd, 1.0), 0.0)

    dhi = Fd * ghi
    dni = (ghi - dhi) / cos_theta

    return {
        "DNI": dni,
        "DHI": dhi,
        "kt": kt,
        "Fd": Fd,
        "Gon": Gon,
        "G0": G0,
        "cos_theta_z": cos_theta,
    }


# -------------------------------------------------
# 4. Sun angles and 3D ENU vectors
# -------------------------------------------------

def solar_angles_and_vectors(lat_deg: float,
                             lon_deg: float,
                             when_local: datetime) -> dict:
    """
    Computes solar zenith, altitude, azimuth and 3D ENU vectors.

    Parameters
    ----------
    lat_deg : float        Latitude in degrees (N positive).
    lon_deg : float        Longitude in degrees (E positive, W negative).
    when_local : datetime  Local wall-clock time at the site, timezone-aware.

    Returns
    -------
    dict with keys:
        'theta_z_deg'   : solar zenith angle (deg)
        'altitude_deg'  : solar altitude (deg)
        'azimuth_deg'   : azimuth from North, positive toward East (deg)
        'sun_vec_enu'   : (x, y, z) unit vector FROM site TO sun
        'ray_vec_enu'   : (x, y, z) unit vector for incoming rays
    """

    # Convert local time
    when_utc = when_local.astimezone(ZoneInfo("UTC"))

    # Basic solar geometry
    n = day_of_year(when_utc)
    delta_deg = solar_declination_deg(n)
    eot = equation_of_time_min(n)

    # UTC as decimal hours
    t_utc = when_utc.hour + when_utc.minute / 60.0 + when_utc.second / 3600.0

    # Local solar time (hours), east longitude positive
    lst = t_utc + lon_deg / 15.0 + eot / 60.0

    # Hour angle ω (deg)
    omega_deg = 15.0 * (lst - 12.0)

    # Zenith angle θz
    phi = math.radians(lat_deg)
    delta = math.radians(delta_deg)
    omega = math.radians(omega_deg)

    cos_theta = (
        math.sin(phi) * math.sin(delta)
        + math.cos(phi) * math.cos(delta) * math.cos(omega)
    )
    cos_theta = max(min(cos_theta, 1.0), -1.0)
    theta_z_deg = math.degrees(math.acos(cos_theta))
    altitude_deg = 90.0 - theta_z_deg

    # Solar azimuth A (0° = North, +East)
    theta = math.radians(theta_z_deg)

    if theta_z_deg >= 90.0:
        # Sun at/below horizon = azimuth not really defined
        az_deg = 0.0
        sun_vec = (0.0, 0.0, 0.0)
        ray_vec = (0.0, 0.0, 0.0)
    else:
        sin_az = -math.sin(omega) * math.cos(delta) / math.sin(theta)
        cos_az = (math.sin(delta) - math.sin(phi) * math.cos(theta)) / (
            math.cos(phi) * math.sin(theta)
        )

        az_rad = math.atan2(sin_az, cos_az)
        az_deg = math.degrees(az_rad)
        if az_deg < 0:
            az_deg += 360.0

        # 3D ENU unit vectors
        h_rad = math.radians(altitude_deg)
        A_rad = math.radians(az_deg)

        # Unit vector FROM site to sun (ENU)
        x_sun = math.cos(h_rad) * math.sin(A_rad)   # East
        y_sun = math.cos(h_rad) * math.cos(A_rad)   # North
        z_sun = math.sin(h_rad)                     # Up

        sun_vec = (x_sun, y_sun, z_sun)

    return {
        "theta_z_deg": theta_z_deg,
        "altitude_deg": altitude_deg,
        "azimuth_deg": az_deg,
        "sun_vec_enu": sun_vec,
    }

### Example

In [4]:
lat = 32.61
lon = -85.48

# Local time with DST handled by the timezone
when_local = datetime(2025, 9, 1, 18, 54, 0, tzinfo=ZoneInfo("America/Chicago"))

# Zenith with DNI/DHI from GHI
when_utc = when_local.astimezone(ZoneInfo("UTC"))
theta_z_deg, cos_theta_z, n = solar_zenith_from_utc(lat, lon, when_utc)

ghi = 9.0  # pyranometer reading, W/m^2
res = decompose_ghi_to_dni_dhi(ghi, theta_z_deg, n)

print("Decomposition of GHI to DNI, DHI:")
print(f"Local time:             {when_local}")
print(f"UTC time:               {when_utc}")
print(f"Day of year, n          = {n}")
print(f"Solar zenith, θz (deg)  = {theta_z_deg:.3f}")
print(f"cos(θz)                 = {res['cos_theta_z']:.4f}")
print(f"Gon (W/m^2)             = {res['Gon']}")
print(f"G0 (W/m^2)              = {res['G0']}")
print(f"k_t                     = {res['kt']}")
print(f"Diffuse fraction, Fd    = {res['Fd']}")
print(f"DHI (W/m^2)             = {res['DHI']:.1f}")
print(f"DNI (W/m^2)             = {res['DNI']:.1f}")

# Sun position & ENU vectors
res_2 = solar_angles_and_vectors(lat, lon, when_local)

print("\nSun angles & 3D ENU vectors:")
print(f"Solar altitude (deg)    = {res_2['altitude_deg']:.3f}")
print(f"Solar azimuth (deg)     = {res_2['azimuth_deg']:.3f}")
print(f"Sun vector (ENU)        = {res_2['sun_vec_enu']}")

Decomposition of GHI to DNI, DHI:
Local time:             2025-09-01 18:54:00-05:00
UTC time:               2025-09-01 23:54:00+00:00
Day of year, n          = 244
Solar zenith, θz (deg)  = 88.500
cos(θz)                 = 0.0262
Gon (W/m^2)             = 1344.8943168287187
G0 (W/m^2)              = 35.21593737239063
k_t                     = 0.25556610647132794
Diffuse fraction, Fd    = 0.9716072217349017
DHI (W/m^2)             = 8.7
DNI (W/m^2)             = 9.8

Sun angles & 3D ENU vectors:
Solar altitude (deg)    = 1.500
Solar azimuth (deg)     = 278.214
Sun vector (ENU)        = (-0.9894030964742863, 0.14281408595474018, 0.026184910540354146)
